# Modellvurdering: Segmentering av snuplasser i flyfoto
# Eksempel 
## Mål
Velge én eller flere arkitekturer for semantisk segmentering av snuplasser i flyfoto. 
Vurderingen baseres på ytelse, krav til input, tilgjengelige pretrente vekter og fleksibilitet.

## Kriterier for modellutvelgelse og vurdering
Før testing definerte vi noen felles kriterier for hvilke modeller som er aktuelle å prøve, og hva vi legger vekt på i vurderingen:


**- God ytelse med lite data:** Modellene bør kunne trenes selv om tilgjengelig treningsdata er begrenset.

**- Rask bildeprosessering:** Det er ønskelig at modellen kan gi raske prediksjoner (f.eks. i felt eller nær sanntid).

**- Åpen kildekode og tilgjengelighet:** Vi prioriterte modeller som er open source og kan lastes ned og brukes umiddelbart uten behov for lisens.

**- Input-størrelse:** Kan modellen ta fleksible dimensjoner? Krever kvadratiske tiles?

**- Pretrained weights:** Finnes det ferdigtrente vekter på passende domene?

**- Treningshastighet og ressursbruk:** Er modellen trenbar lokalt eller krever den kraftig maskin?

**- Generaliserbarhet:** Takler modellen variasjon i terreng og belysning?

**- Modifiserbarhet:** Er arkitekturen lett å endre på for fremtidige behov?





## Oversikt over aktuelle modeller

| Modell        | Arkitekturtype     | Pretrained tilgjengelig | Typisk input-størrelse | Vekt  | Kommentar |
|---------------|--------------------|---------------------------|---------------------------|--------|-----------|
| **U-Net**     | CNN (encoder-decoder) | Delvis (f.eks. via `segmentation_models.pytorch`) | Fleksibel (ofte 256–512) | Lav    | God baseline, enkel, lett å modifisere |
| **DeepLabV3** | CNN + ASPP         | Ja (COCO, ImageNet via `torchvision`) | 512x512 eller større     | Medium | Bra på detaljer og objektkanter |
| **SwinUNet**  | Vision Transformer + U-Net | Ja (medisinsk, via HUGGINGFACE) | Kvadratisk (224–512) | Høy    | Bra ytelse, tregere trening, nyere |
| **UNet++**    | Forbedret U-Net    | Delvis                    | 256–512                   | Medium | Bedre på små detaljer, flere hoppforbindelser |
| **HRNet**     | Multi-resolusjon   | Ja                        | 512+                      | Høy    | Bra for detaljer, men kompleks |
| **DIVE-SCI**  | Transformer(domain-adaptiv) |Ja(spesialisert for klima/satellitt)| 256-512 | Medium | Tilpasset domene, god ved lite data, vurderes ikke testet i denne runden|
| **SAM**       | Foundation model/ Prompt- basert| Ja(Meta AI, open source) | 1024+ | Høy | Krever prompt, ikke direkte sammenlignbar med klassiske segmenteringsmodeller|



## Hvorfor disse modellene – og hvorfor ikke de andre?
### Valgte modeller
**+ U-Net:** 
U-Net er et fullt konvolusjonelt nevralt nettverk utviklet spesielt for bildesegmentering. Det har vist seg å fungere godt selv med små treningsdatasett, noe som gjør det godt egnet i situasjoner hvor annotert data er begrenset. En av hovedfordelene er at det kan behandle et bilde på under ett sekund når det kjøres på en GPU, noe som gjør det effektivt også for sanntidsbruk.

U-Net er åpen kildekode og fritt tilgjengelig, noe som betyr at hvem som helst kan laste det ned og ta det i bruk uten lisensbegrensninger. Nettverket består kun av konvolusjonslag, uten noen fullt tilkoblede (fully connected) lag, og dette gjør det bedre egnet til oppgaver hvor bevaring av romlig informasjon er viktig.

Modellen er bygget opp slik at den fanger opp lokale mønstre i bildet, og undersøker forholdet mellom hver piksel og dens omgivelser. Den kombinerer lavnivå og høynivå egenskaper gjennom såkalte hoppkoblinger, noe som bidrar til mer presis segmentering. Konvolusjonslagene reduserer kanskje størrelsen på bildet noe, men de bevarer den posisjonelle informasjonen – avgjørende for nøyaktige segmenteringsgrenser.

I evalueringer har U-Net vist lav rank error og lav warping error, noe som tyder på at modellen er god til å gruppere strukturer riktig og tegne segmenteringskanter på de riktige stedene.

|       Rank      | Group Name | Warping Error | Rand Error | Pixel Error |
|-----------------|------------|---------------|------------|-------------|
|                 | **human values**| 0.000005 | 0.0021     | 0.0010      |
|        1.       |  ***U-Net***    |***0.000353*** |***0.0382***     | ***0.0611***      |
|        2.       |  DIVE-SCI       | 0.000355 | 0.0305     | 0.0611      |
|        3.       |  IDSIA          | 0.000420 | 0.0504     | 0.0613      |
|        4.       |  DIVE           | 0.000430 | 0.0545     | 0.0582      |
|        .        |    .            |     .    |     .      |     .       |
|        .        |    .            |     .    |     .      |     .       |
|        .        |    .            |     .    |     .      |     .       | 
|        10.      | IDSIA-SCI       | 0.000653 | 0.0189     | 0.1027      |

-Rangering i EM-segmenteringsutfordringen(6.mars 2015), sortert etter warping- feil.

Dette passer godt med behovene i vårt prosjekt, hvor vi jobber med relativt lite annotert 
treningsdata og samtidig har behov for rask og effektiv prediksjon i store bildesett.
U-Nets evne til å levere nøyaktige segmenteringskanter og gruppere relevante strukturer på 
en robust måte gjør den godt egnet for oppgaven med å identifisere snuplasser i flyfoto. 
Derfor valgte vi å inkludere U-Net som en av hovedmodellene i vår første testfase.

**+ DeepLabV3:**
DeepLabV3 er en avansert segmenteringsmodell som kombinerer en kraftig encoder–decoder-arkitektur med atrous (dilated) convolutions for å oppnå høy presisjon i komplekse bildeomgivelser. Modellen har vist seg å håndtere utfordringer som uklare objekter, svake markeringer, varierende lysforhold, skygger og ujevne bakgrunner – forhold som også er relevante i vårt domene med flyfoto.

Flere teknikker bidrar til modellens robusthet og nøyaktighet:

**Dataforsterkning (augmentation):** DeepLabV3 drar nytte av ulike augmenteringsteknikker for å øke variasjonen i treningsdataene, noe som er spesielt nyttig når tilgjengelig data er begrenset.

**Tilpasning til ubalanserte klasser:** Modellen støtter bruk av tilpassede tapsfunksjoner som Dice Loss og Jaccard Loss, hvor klassene vektes i henhold til forekomsten i datasettet. Dette forbedrer nøyaktigheten for minoritetsklasser.

**Fleksibel arkitektur:** DeepLabV3 kan benytte flere ulike backbone-nettverk (som VGG16, VGG19, ResNet-50 og ResNet-101), noe som gir mulighet for å tilpasse modellen til tilgjengelige ressurser og krav til ytelse.

**Transfer learning:** Ved å bruke forhåndstrente vekter oppnås gode resultater selv med lite treningsdata, noe som gjør modellen aktuell i tidlige faser av et prosjekt.

**Sterk på nøyaktige metrikker:** I flere studier har DeepLabV3 vist bedre resultater enn andre metoder når det gjelder presisjon, recall og F1-score.

Disse egenskapene gjør modellen godt egnet for scenarier med høy visuell kompleksitet, slik som identifisering av snuplasser i flyfoto, der klare segmenteringskanter og robusthet mot variasjon i bildedata er avgjørende.

**Segment Anything Model (SAM):**
SAM er en ny og kraftig segmenteringsmodell utviklet av Meta AI. Den er trent på over 1 milliard segmenteringsmasker og fungerer etter prompt-basert prinsipp: brukeren gir et punkt, boks eller tekst som forteller hva som skal segmenteres. Dette gjør modellen ekstremt fleksibel og i stand til å segmentere objekter den aldri har sett før (zero-shot). Den fungerer godt på ulike typer bilder – også flyfoto – uten at man trenger egen trening. I tillegg er den åpen kildekode, rask, og lett å integrere i interaktive verktøy for bildeannotering.

I vårt prosjekt vurderer vi SAM som en referansemodell på grunn av dens generaliseringsevne, fleksibilitet og nytteverdi ved manuell gjennomgang. Den inngår ikke i batch-trening på samme måte som U-Net og DeepLabV3+, men er svært aktuell for støtte i kvalitetskontroll eller som del av en kombinert pipeline.


## Modeller vi vurderte, men valgte bort

Flere modeller ble vurdert i den tidlige fasen av prosjektet, men ble ikke valgt for første runde med testing.

**SwinUNet :** Det ble vurdert som en lovende modell, spesielt på grunn av sin bruk av transformerarkitektur kombinert med U‑Net‑struktur. Likevel ble den valgt bort på grunn av høy treningskostnad og krav til GPU-ressurser. Vi prioriterte raskere og lettere modeller i første omgang.

**DIVE-SCI :** Det er spesialutviklet for satellitt‑ og klimadata, med fokus på domeneadapsjon. Den er ikke nødvendigvis optimal for våre flyfoto‑baserte data, og ble derfor vurdert som mindre relevant for dette prosjektet.

**UNet++ :** UNet++ er en videreutvikling av U-Net med flere hoppkoblinger og tettere forbindelser mellom encoder og decoder. Selv om modellen er kjent for å forbedre segmentering av små detaljer, vurderer vi at den ikke gir en vesentlig forskjell sammenlignet med standard U-Net i vår kontekst. Siden vi allerede tester U-Net, som har samme grunnstruktur, oppfatter vi det som unødvendig å teste en så nært beslektet variant separat. Ved å fokusere på U-Net dekker vi i praksis også mye av det UNet++ representerer, og vi slipper å bruke tid og ressurser på en modell med begrenset tilleggseffekt i denne fasen.


## Mulighet for å kombinere modeller

- **Ensemble**: Flere modeller stemmer overens (snitt, majority vote)
- **Kombinert pipeline**: F.eks. en CNN for grov lokalisering + transformer for fin segmentering
- **Stage-basert**: Bruke en modell for bounding box, og en annen for mask

---

## Neste steg
- Test en baseline-modell (U-Net, SAM eller DeepLabV3)
- Evaluer nøyaktighet og ressursbruk på et mindre testsett
- Vurder om mer avanserte modeller gir gevinst i presisjon/robusthet

---
